In [ ]:
# ENV["PYTHON"]= "/kuacc/users/oulas15/.conda/envs/oulas15/bin/python"
# using Pkg
# Pkg.build("PyCall")

In [1]:
using Knet, CUDA, MLDatasets, ArgParse, Images, Random, Statistics,Base, Plots,NPZ
#include(Knet.dir("data","imagenet.jl"))
const F = Float32
Atype = CUDA.functional() ? KnetArray{F} : Array{F}

KnetArray{Float32, N} where N

In [2]:
data_mean,data_std = 23.206272901271582, 42.44736819464951

(23.206272901271582, 42.44736819464951)

In [3]:
test_data = npzread("../Mouse_n0/test/test_data.npz")

Dict{String, Array{T, 3} where T} with 2 entries:
  "Y_test" => UInt16[0x0000 0x0000 … 0x0000 0x0000; 0x0000 0x0000 … 0x0000 0x00…
  "X_test" => UInt8[0x00 0x00 … 0x01 0x01; 0x01 0x00 … 0x00 0x00; … ; 0x01 0x01…

In [4]:
xtst=test_data["X_test"];
xtst=permutedims(xtst, [2, 3, 1]);

In [5]:
test_noisy = Array{Float64}(xtst) + Array(randn(Float64,size(xtst))*(20));

In [6]:
batch_size = 1
dtst = minibatch(test_noisy,xtst, 1; xsize = (256, 256, 1,:),ysize = (256, 256, 1,:), xtype = Atype,ytype=Atype);

In [7]:
function encode(ϕ, x)
    x = conv4(ϕ[1], x, padding=1)
    x = relu.(x .+ ϕ[2])
    x = conv4(ϕ[3], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[4])
    x = pool(x, stride = 2, mode=0)
  
    x = conv4(ϕ[5], x, padding=1)
    x = relu.(x .+ ϕ[6])
    x = conv4(ϕ[7], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[8])
    x = pool(x, stride = 2, mode=0)
 
    x = conv4(ϕ[9], x, padding=1)
    x = relu.(x .+ ϕ[10])
    x = conv4(ϕ[11], x, padding=1, stride=1)
    x = relu.(x .+ ϕ[12])
    x = pool(x, stride = 2, mode=0)

    
    mu = conv4(ϕ[13], x, padding=1, stride=1)
    mu = mu .+ ϕ[14]
    
    logvar = conv4(ϕ[15], x, padding=1, stride=1)
    logvar = logvar .+ ϕ[16]
    
    return mu, logvar
end

function decode(θ, z)
    z = conv4(θ[1], z, padding=1)
    z = relu.(z .+ θ[2])
    z = conv4(θ[3], z, padding=1, stride=1)
    z = relu.(z .+ θ[4])
    z = deconv4(θ[5], z, padding=0, stride=2)
    z = relu.(z .+ θ[6])
  
    z = conv4(θ[7], z, padding=1)
    z = relu.(z .+ θ[8])
    z = conv4(θ[9], z, padding=1, stride=1)
    z = relu.(z .+ θ[10]) 
    z = deconv4(θ[11], z, padding=0, stride=2)
    z = relu.(z .+ θ[12])
    
    z = conv4(θ[13], z, padding=1)
    z = relu.(z .+ θ[14])
    z = conv4(θ[15], z, padding=1, stride=1)
    z = relu.(z .+ θ[16]) 
    z = deconv4(θ[17], z, padding=0, stride=2)
    z = relu.(z .+ θ[18])
    
    z = conv4(θ[19], z, padding=1)
    z = z .+ θ[20]
    return z
end

decode (generic function with 1 method)

In [8]:
function MSE(x, y,)
    return mean((x - y).^2)
end

MSE (generic function with 1 method)

In [9]:
function reparameterize(mu,logvar)
    std = exp.(logvar.*0.5)
    epsilon = convert(Atype, randn(F, size(mu)))
    z = mu .+ (epsilon.*std)
    return z 
end

reparameterize (generic function with 1 method)

In [10]:
w2 = Knet.load("best_aug.jld2","w",)

Dict{Symbol, Vector{KnetArray{Float32, 4}}} with 2 entries:
  :decoder => [K32(3,3,64,128)[0.00014185027⋯], K32(1,1,128,1)[0.36012068⋯], K3…
  :encoder => [K32(3,3,1,32)[0.022676684⋯], K32(1,1,32,1)[0.00011777959⋯], K32(…

In [11]:
function inference_forward(w,x,y_hat,data_mean,data_std,n)
    wdec, wenc = w[:decoder], w[:encoder]
    x_normalized = (x .- data_mean)./ data_std
    mu, logvar = encode(wenc, x_normalized)
    y_total = Atype(zeros(Float32,size(x)))
    for i in 1:n
        z = reparameterize(mu,logvar)
        y = decode(wdec,z)
        y  = (y.*data_std) .+ data_mean
        y_total = y_total .+ y
    end
    y_total = y_total./n
    return MSE(y_hat,y_total)
end

inference_forward (generic function with 1 method)

In [13]:
PSNR_total = 0
for (i,x)  in enumerate(dtst)
    noisy,y = x
    mse = inference_forward(w2,noisy,y,data_mean,data_std,1000)
    PSNR_total += 10*Base.log10((255*255)/mse)
end

In [14]:
PSNR_total/length(dtst)

34.08879f0